In [ ]:
a, b, *rest = list(range(5))
a, b, rest

In [1]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)), #
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas: #
    if longitude <= 0: #
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


## Named Tuples   
The collections.namedtuple function is a factory that produces subclasses of tuple  
enhanced with field names and a class name—which helps debugging.  
```
Instances of a class that you build with namedtuple take exactly the  
same amount of memory as tuples because the field names are  
stored in the class. They use less memory than a regular object  
because they don’t store attributes in a per-instance __dict__.  
```


In [2]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates'.split())
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

A named tuple type has a few attributes in addition to those inherited from tuple.  
Most useful: the `_fields` class attribute, the class method `_make(iterable)`, and the `_asdict()` instance method.  

- `_fields` is a tuple with the field names of the class.
- `_make()` allow you to instantiate a named tuple from an iterable; City(*del
hi_data) would do the same.
- `_asdict()` returns a collections.OrderedDict built from the named tuple
instance. That can be used to produce a nice display of city data

In [3]:
print(City._fields)
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()
for key, value in delhi._asdict().items():
    print(key + ':', value)

('name', 'country', 'population', 'coordinates')
name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)


`t: some tuple`
- t.__getnewargs__(): Support for optimized serialization with pickle (doesn't support for list)

In [4]:
t = (1,2)
t *= 2
t

(1, 2, 1, 2)

## Slice Objects
```
s = 'bicycle'
s[::-1]
'elcycib'
```

```
invoice = """
1909 Pimoroni PiBrella $17.50 3 $52.50
1489 6mm Tactile Switch x20 $4.95 2 $9.90
1510 Panavise Jr. - PV-201 $28.00 1 $28.00
1601 PiTFT Mini Kit 320x240 $34.95 1 $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])
```

## Assigning to Slices
Mutable sequences can be grafted, excised, and otherwise modified in place using slice  
notation on the left side of an assignment statement or as the target of a del statement.

In [5]:
l = list(range(10))
l
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
l[2:5] = [20, 30]  # [0, 1, 20, 30, 5, 6, 7, 8, 9]
del l[5:7]  # [0, 1, 20, 30, 5, 8, 9]
l[3::2] = [11, 22]  # [0, 1, 20, 11, 5, 22, 9]
# l[2:5] = 100  # # TypeError: can only assign an iterable
l[2:5] = [100]  # [0, 1, 100, 22, 9]
l

[0, 1, 100, 22, 9]

The `ellipsis` written with three full stops `...` is recognized as a token by the Python parser.   
It is an alias to the `Ellipsis` object, the single instance of the ellipsis class.2   
As such, it can be passed as an argument to functions and as part of a slice specification,   
as in `f(a, ..., z)` or `a[i:...]`. NumPy uses `...`
as a shortcut when slicing arrays of many dimensions; for example, if x is a `4` dimensional
array, `x[i, ...]` is a shortcut for `x[i, :, :, :,]`

In [6]:
# pitfalls of trying to use * to initialize a list of lists.
l = [[0]] * 3
# l[0][0] = 0
l[0][0] = 3
l

[[3], [3], [3]]

## Augmented Assignment with Sequences
Augmented assignment operators `+=` and `*=`   
For `list` `a += b` will work as `a.extend(b)`  
For `list` (mostly multable objects) `+=` and `*=` doesn't create new object.  
Like `a = a + b` will create a new object then assign it ti `a`

In [7]:
# Mutable List
l = [1,2,3]
print(id(l))
l+=[1,2]
print(id(l))
l = l + [1,2]
print(id(l))

1683944416128
1683944416128
1683944789568


In [8]:
# Mutable Tuple
l = (1,2,3)
print(id(l))
l+=(1,2)
print(id(l))

1683944399360
1683944459936


In [9]:
# Here we see an error cause tuples are immutable but still it's value is changed
# Lessons learned:
# - Putting mutable items in tuples is not a good idea.
# - Augmented assignment is not an atomic(independent) operation, we just saw it throwing an
#   exception after doing its job.
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [10]:
print(t)

(1, 2, [30, 40, 50, 60])


In [11]:
# Bytecode for some expression
import dis
print(dis.dis('a=b'))
print("-----")
print(dis.dis('s[a] += b'))

  1           0 LOAD_NAME                0 (b)
              2 STORE_NAME               1 (a)
              4 LOAD_CONST               0 (None)
              6 RETURN_VALUE
None
-----
  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None


- Important Python API convention: functions or methods that change an object in
place should return None to make it clear to the caller that the object itself was changed,
and no new object was created

In [12]:
import bisect
l = [0, 1, 2, 5, 6, 33, 45, 56]
print(bisect.bisect(l, 7))
print(bisect.bisect_left(l, 7))
bisect.insort(l, 3)
l

5
5


[0, 1, 2, 3, 5, 6, 33, 45, 56]

## Array

As of Python 3.4, the array type does not have an in-place sort method like list.sort().  
We can use the sorted function to rebuild it:  
`a = array.array(a.typecode, sorted(a))`   
To keep a sorted array sorted while adding items to it, use the `bisect.insort` function

In [13]:
from array import array
floats = array('d', [1, 2, 3, 4])  # array('d', [1.0, 2.0, 3.0, 4.0])
fp = open('floats.bin', 'wb')
floats.tofile(fp)
fp.close()
floats2 = array('d')
fp = open('floats.bin', 'rb')
floats2.fromfile(fp, 4)
fp.close()

In [14]:
numbers = array('h', [-2, -1, 0, 1, 2])  # 'h': signed short (2 bytes)
memv = memoryview(numbers)
memv_oct = memv.cast('B')  # 'B': unsigned char (1 byte)
print(memv_oct.tolist())  # Resultent is 2*sizeof(numbers) cause of (signed short (2 bytes) -> unsigned char (1 byte))
memv_oct[5] = 4
print()
print(f"numbers: {numbers}")
print(f"memv: {memv.tolist()}")
print(f"memv_oct: {memv_oct.tolist()}")

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

numbers: array('h', [-2, -1, 1024, 1, 2])
memv: [-2, -1, 1024, 1, 2]
memv_oct: [254, 255, 255, 255, 0, 4, 1, 0, 2, 0]


## Deques and Other Queues

The `append` and `popleft` operations are atomic(independent), so `deque` is safe to use as a LIFO queue in `multithreaded` applications without the need for using locks.

In [15]:
from collections import deque
dq = deque(range(10), maxlen=10)
print(dq)
dq.rotate(3)  # pick form last and add to first
print(f"Rotate 3: {dq}")
dq.rotate(-4)  # pick form first and add to last
print(f"Rotate -4: {dq}")
dq.appendleft(-1)
print(f"Append Left -1: {dq}")
dq.extend([11, 22, 33])  # Add to right
print(f"Extend: {dq}")
dq.extendleft([10, 20, 30, 40])
print(f"Extend Left: {dq}")

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
Rotate 3: deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
Rotate -4: deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
Append Left -1: deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
Extend: deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33], maxlen=10)
Extend Left: deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8], maxlen=10)


## queue
This provides the synchronized (i.e., `thread-safe`) classes `Queue`, `LifoQueue`, and
`PriorityQueue`.   
These are used for safe communication between threads.  
They don’t discard items to make room as deque does.   
Instead, when the queue is full the insertion of a new item, it waits until
some other thread makes room by taking an item from the queue (which is useful
to throttle the number of live threads).
## multiprocessing
Implements its own bounded `Queue`, very similar to queue.Queue but designed for
interprocess communication.   
A specialized `multiprocessing.JoinableQueue` is also available for easier task management.
## asyncio
Newly added to Python 3.4, asyncio provides `Queue`, `LifoQueue`, `PriorityQueue`, and `JoinableQueue` with APIs inspired by the classes contained in the queue and multiprocessing modules, but adapted for managing tasks in asynchronous programming.
## heapq
In contrast to the previous three modules, `heapq` does not implement a `queue` class,
but provides functions like `heappush` and `heappop` that let us use a mutable sequence
as a heap queue or priority queue.